## Robot@Home 2 - Characterized objects & observations `v1.2`

`R@H2 notebook series`   

<a href="https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/80-Characterized-observations.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>



### R@H2 functions introduced in this notebook:



-   `rh.get_observations()`
-   `rh.get_objects_in_observation(id)`
-   `rh.get_objects()`
-   `rh.get_object_relations(id)`



### Initial requirements



Install Robot@Home2 Toolbox using the Python package manager



In [1]:
!pip install robotathome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 836 kB 7.7 MB/s 
     |████████████████████████████████| 58 kB 3.3 MB/s 


Check that the installation has been successful



In [2]:
import robotathome as rh
print (rh.__version__)

1.0.5


#### Using Google Colab and Google Drive



If you haven't yet mounted R@H2 Dataset public share on your Google Drive click
[this](https://drive.google.com/drive/folders/15fQwm4G3hHQR5vI9q0CVDlwxMwaJheC8?usp=sharing).

In order to access from Google Colab it's mandatory to create a `shorcut`. You
just need to select the shared folder `Shared with me > R@H2-2.0.1`, right click
and select the corresponding menu option `Add Shortcut to Drive`. Now, a new
folder (actually a shortcut) with the same name will appear under `My Drive`.

Next, mount Google Drive in Google Colab. The browser will ask for the
authorization to access the Google Drive account. Sign in to your Google account
and Google Drive will be mounted to `/content/drive`. Note that the R@H2 files are
located in the `/content/drive/MyDrive/R@H2-2.0.1` folder.



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


More info in [this notebook](https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/05-Google-colab-drive.ipynb).



### Importing R@H2



In [4]:
import os
import pandas as pd
from robotathome import RobotAtHome
from robotathome import logger, log, set_log_level

In [5]:
log.set_log_level('INFO')  # SUCCESS is the default

level_no, level_name = log.get_current_log_level()
print(f'Current log level name: {level_name}')

Current log level name: INFO


### Instantiating the toolbox



The toolbox is modeled around the RobotAtHome class, so we'll begin instantiating it and
creating a working object.

The RobotAtHome class has some attributes:

-   `rh_path` : string with the full path for robotathome database, usually `rh.db`.
    Its default value is `'.'` (the current directory)
-   `wspc_path` : string with the full path where results or temporary files are
    stored. It's the path to the workspace directory. Its default value also is
    `'.'`.
-   `db_filename` : string with the database filename. Its default value is `'rh.db'`
-   `rgbd_path` : string with the full path to where RGBD images are stored. Its default value is
    `'./files/rgbd'`.
-   `scene_path` : string with the full path to where Scene (3D) images are stored. Its
    default value is `'./files/scene'`.

We assume that we are running this notebook in Google Colab. Therefore our
folder structure is as follows:

    /content/drive
    └─── MyDrive
         ├─── R@H2-2.0.1
         │    └── files
         │        ├── rgbd
         │        └── scene
         └─────── rh.db

Let's instantiate our first, and usually the only one, object:



In [6]:
my_rh_path = '/content/drive/MyDrive/R@H2-2.0.1'
my_rgbd_path = os.path.join(my_rh_path, 'files/rgbd')
my_scene_path = os.path.join(my_rh_path, 'files/scene')
my_wspc_path = '/content'

try: 
      rh = RobotAtHome(my_rh_path, my_rgbd_path, my_scene_path, my_wspc_path)
except:
      logger.error("Something was wrong")

2022-06-05 12:02:08.079 | SUCCESS  | robotathome.core.reader:__open_dataset:85 - Connection is established: rh.db


Now you are ready to dive into the R@H2 database. Following we'll show a set of
functions to extract some of the main reference data.



### Introduction



Robot@Home2 Dataset contains the features extracted from the difference objects
and rooms within each scene. In addition, the dataset contains the featured
relations among objects.



### Observations



Specifically, for rooms there is a set of RGBD observations that has been
characterized considering averaged features from past observations, as well as
features of the closest laser scan in time. The number of room features is 48
and the number of scan features is 9.

Unfortunately, this set of observations should be considered experimental in the
sense that the timestamp data has been lost, preventing the identification of
the sensor observations from which they where extracted. Nevertheless,
RGBD observations are sequentially ordered.



In [7]:
observations = rh.get_observations()
logger.info("\nObservations: \n{}", observations.info())

2022-06-05 12:02:18.765 | INFO     | __main__:<module>:2 - 
Observations: 
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9608 entries, 0 to 9607
Data columns (total 63 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           9608 non-null   int64  
 1   room_id                      9608 non-null   int64  
 2   home_id                      9608 non-null   int64  
 3   home_session_id              9608 non-null   int64  
 4   home_subsession_id           9608 non-null   int64  
 5   sensor_id                    9608 non-null   int64  
 6   mean_hue                     9608 non-null   float64
 7   mean_saturation              9608 non-null   float64
 8   mean_value                   9608 non-null   float64
 9   hue_stdv                     9608 non-null   float64
 10  saturation_stdv              9608 non-null   float64
 11  value_stdv                   9608 non-null   float64
 12  hue_histogram_1              9608 non-null   float64
 13  hue_histogram_2   

In [8]:
# For example, let's query the observations dataframe with the id value equal to 10315
obs_id = 10315
observation = observations.query(f'id=={obs_id}')
logger.info('\nObservation with id : {} \n{}', obs_id, observation.iloc[0])

2022-06-05 12:02:23.694 | INFO     | __main__:<module>:4 - 
Observation with id : 10315 
id                    10315.000000
room_id                  40.000000
home_id                   4.000000
home_session_id           6.000000
home_subsession_id        1.000000
                          ...     
scan_num_of_points      245.000000
scan_compactness         21.129900
scan_compactness2         2.341920
scan_linearity            0.657488
scan_scatter              0.828744
Name: 9607, Length: 63, dtype: float64


### Objects



The objects extracted from the observations have also been characterized. For
each object the number of features is 32



In [9]:
objects = rh.get_objects()
objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2125 entries, 0 to 2124
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2125 non-null   int64  
 1   room_id                 2125 non-null   int64  
 2   home_id                 2125 non-null   int64  
 3   home_session_id         2125 non-null   int64  
 4   home_subsession_id      2125 non-null   int64  
 5   name                    2125 non-null   object 
 6   object_type_id          2125 non-null   int64  
 7   planarity               2125 non-null   float64
 8   scatter                 2125 non-null   float64
 9   linearity               2125 non-null   float64
 10  min_height              2125 non-null   float64
 11  max_height              2125 non-null   float64
 12  centroid_x              2125 non-null   float64
 13  centroid_y              2125 non-null   float64
 14  centroid_z              2125 non-null   

Remember that a subset of objects can be extracted filtering the resulting dataframe



In [10]:
# Let's look for objects in a room
r_name = 'alma_kitchen1'
r_id  = rh.name2id(r_name,'r')
objects_in_room = objects.query(f'room_id=={r_id}')
logger.info("\nObjects in room {} (id:{}) : \n{}", r_name, r_id, objects_in_room['name'])

2022-06-05 12:02:48.990 | INFO     | __main__:<module>:5 - 
Objects in room alma_kitchen1 (id:3) : 
40        wall_cabinet_0
41        refrigerator_0
42           microwave_0
43       upper_cabinet_0
44       upper_cabinet_1
45     smoke_extractor_0
46       upper_cabinet_2
47    cooking_utensils_0
48              bottle_0
49              dishes_0
50               stove_0
51                 pot_0
52               cloth_0
53              bottle_1
54                bowl_0
55              window_0
56             picture_0
57          paper_roll_0
58                sink_0
59              faucet_0
60              bottle_3
61             cabinet_0
62             cabinet_1
63     washing_machine_0
64             cabinet_2
65                wall_0
66                wall_2
67                wall_3
68               floor_0
69             counter_0
70             counter_1
71              bottle_2
Name: name, dtype: object


#### Relations between objects



The relation between objects has also been calculated and characterized with 11
features



In [11]:
object_relations = rh.get_object_relations()
object_relations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39080 entries, 0 to 39079
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        39080 non-null  int64  
 1   room_id                   39080 non-null  int64  
 2   home_id                   39080 non-null  int64  
 3   home_session_id           39080 non-null  int64  
 4   home_subsession_id        39080 non-null  int64  
 5   obj1_id                   39080 non-null  int64  
 6   obj2_id                   39080 non-null  int64  
 7   minimum_distance          39080 non-null  float64
 8   perpendicularity          39080 non-null  float64
 9   vertical_distance         39080 non-null  float64
 10  volume_ratio              39080 non-null  float64
 11  is_on                     39080 non-null  int64  
 12  abs_hue_stdv_diff         39080 non-null  float64
 13  abs_saturation_stdv_diff  39080 non-null  float64
 14  abs_va

There is also a function to get the relationships an object is implicit in.



In [12]:
# Let's look for the relations the object
o_name = 'microwave_0'
o_id  = rh.name2id(o_name,'o')
object_relations = rh.get_object_relations(o_id)
logger.info("\nThe {} object relations: \n{}", o_id, object_relations[['obj1_id', 'obj2_id']])

2022-06-05 12:03:30.001 | INFO     | __main__:<module>:5 - 
The 44 object relations: 
    obj1_id  obj2_id
0        44       45
1        44       46
2        44       47
3        44       48
4        44       49
5        44       50
6        44       51
7        44       52
8        44       53
9        44       54
10       44       55
11       44       56
12       44       57
13       44       58
14       44       59
15       44       60
16       44       61
17       44       62
18       44       63
19       44       64
20       44       65
21       44       66
22       44       67
23       44       68
24       44       69
25       44       70
26       44       71
27       44       72
28       44       73
29       42       44
30       43       44


#### Objects in observations



Finally, it's possible to extract the objects contained in a characterized RGBD observation



In [13]:
objects_in_observation = rh.get_objects_in_observation(obs_id)
logger.info("\nObjects in observation with id = {} : \n{}", obs_id , objects_in_observation['name'])

2022-06-05 12:03:36.058 | INFO     | __main__:<module>:2 - 
Objects in observation with id = 10315 : 
0        wall_2
1        wall_3
2       floor_0
3       couch_2
4       table_2
5       chair_0
6       chair_1
7       chair_2
8        door_0
9         can_0
10    ceiling_0
Name: name, dtype: object


In [14]:
logger.info("\n{}", objects_in_observation.loc[1])

2022-06-05 12:03:51.731 | INFO     | __main__:<module>:1 - 
id                            2362
room_id                         40
home_id                          4
home_session_id                  6
home_subsession_id               1
name                        wall_3
object_type_id                  13
planarity                 0.153936
scatter                   0.840019
linearity                  0.68306
min_height               -0.009182
max_height                 1.99289
centroid_x               -0.537918
centroid_y                 1.21247
centroid_z                0.381797
volume                     3.50972
biggest_area                7.6681
orientation                89.1977
hue_mean                   73.6967
saturation_mean            0.16206
value_mean                0.742442
hue_stdv                    107.01
saturation_stdv           0.240689
value_stdv                0.216117
hue_histogram_0           0.811557
hue_histogram_1           0.003236
hue_histogram_2           0.02

<a href="https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/80-Characterized-observations.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

